In [4]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from collections import OrderedDict


In [5]:
torch.__version__

'1.3.1'

## 读入(这段读的是aigame作业的下发json)

In [6]:
all_json = []

aigame_botid = ["6048fc6b81fb3b738e911e3b",
               "6048fcf381fb3b738e912cb8",
               "6048fd3781fb3b738e9138ac",
               "6048fd7981fb3b738e9140fc",
               "6048fda981fb3b738e914488"]

def get_all_json(cwd):
    cur_dir = os.listdir(cwd)  
    for i in cur_dir:
        sub_dir = os.path.join(cwd, i)
        if os.path.isdir(sub_dir):
            get_all_json(sub_dir)
        else:
            if i[-5:] == ".json":
                all_json.append(cwd + "/" + i)
def check_nb(bot_id):
    return bot_id == aigame_botid[2]
                
get_all_json("../data")

## 读入(这段读的是botzone上下载的对局数据，nb_bots是天梯上排名前30的botid)

## 预处理 Combo

In [7]:
def getCardId(card):
    # 求一张牌的 id
    if card < 52:
        return card // 4
    else:
        return card - 39

def getCombo(cards):
    # a combo is represented as a tuple(k, l, r, w)
    # 表示有 k * [l, r] 即 k 张 [l, r] 中的牌（作为主体）w \in [0, 1, 2] 表示带的是啥类型
    if len(cards) == 0:
        return (0, 0, 0, 0)
    tmp = np.zeros(15, dtype = int)
    for card in cards:
        tmp[getCardId(card)] += 1
    k = np.max(tmp)
    l = np.min(np.where(tmp == k))
    r = np.max(np.where(tmp == k))
    w = 0
    if k == 3:
        w = len(cards) // (r - l + 1) - 3
    if k == 4:
        w = (len(cards) // (r - l + 1) - 4) // 2
    return (k, l, r, w)

combo_dict = {}
combo_list = []
combo_cnt = 0

def initCombo():
    global combo_dict, combo_list, combo_cnt
    combo_dict = {}
    combo_list = []
    combo_cnt = 0
    def addCombo(combo):
        global combo_dict, combo_list, combo_cnt
        combo_list.append(combo)
        combo_dict[combo] = combo_cnt
        combo_cnt += 1

    minLength = [0, 5, 3, 2, 2]
    maxWings = [0, 1, 1, 3, 3]
    fold = [0, 0, 0, 1, 2]
    for k in range(1, 5):
        for x in range(13):
            for w in range(maxWings[k]):
                addCombo((k, x, x, w))
        for l in range(12):
            for r in range(l + minLength[k] - 1, 12):
                for w in range(maxWings[k]):
                    if (r - l + 1) * (k + w * fold[k]) <= 20:
                        addCombo((k, l, r, w))
    addCombo((1, 13, 13, 0))
    addCombo((1, 14, 14, 0))
    addCombo((1, 13, 14, 0))
    addCombo((0, 0, 0, 0))
    
initCombo()

def getPartition(cards):
    # 把一次出牌的编号集合划分成 mainbody 和 bywings
    # 其中 mainbody 是一个 list ，bywings 中每个 wing 是一个 list ，也就是一个 list 的 list
    combo = getCombo(cards)
    tmp = [[] for i in range(15)]
    for card in cards:
        tmp[getCardId(card)].append(card)
    mainbody, bywings = [], []
    for i in range(15):
        if len(tmp[i]) > 0:
            if combo[1] <= i and i <= combo[2]:
                mainbody.extend(tmp[i])
            else:
                bywings.append(tmp[i])
    return mainbody, bywings

def getComboMask(combo):
    # 给出一个 combo ，返回可以接在其后面牌型 mask 
    mask = np.zeros(combo_cnt)
    if combo == (0, 0, 0, 0):
        mask = np.ones(combo_cnt)
        mask[combo_dict[(0, 0, 0, 0)]] = 0
        return mask
    mask[combo_dict[(0, 0, 0, 0)]] = 1

    if combo == (1, 13, 14, 0):
        return mask
    mask[combo_dict[(1, 13, 14, 0)]] = 1

    if combo[0] == 4 and combo[1] == combo[2] and combo[3] == 0:
        for i in range(combo[1] + 1, 13):
            mask[combo_dict[(4, i, i, 0)]] = 1
        return mask
    for i in range(13):
        mask[combo_dict[(4, i, i, 0)]] = 1

    for cb in combo_list:
        if cb[0] == combo[0] and cb[2] - cb[1] == combo[2] - combo[1] and cb[3] == combo[3] and cb[1] > combo[1]:
            mask[combo_dict[cb]] = 1
            
    return mask


In [30]:
_input_size = 0

class Game(object):
    # 这里 0 始终是地主，1 始终是地主下家，2 始终是地主上家

    def __init__(self, init_data):
        self.hand = np.zeros((3, 15), dtype = int)
        for player in range(3):
            for card in init_data[player]:
                self.hand[player, getCardId(card)] += 1
        self.initial_hand = self.hand.copy()
    
    def play(self, player, cards):
        # 模拟打牌 打出 cards 这个 list 中的所有牌
        for card in cards:
            self.hand[player, getCardId(card)] -= 1
            
    def possess(self, player, combo):
        # 判断 player 这个玩家是否拥有 combo 这个牌型的牌
        if combo == (0, 0, 0, 0):
            return True
        for i in range(combo[1], combo[2] + 1):
            if self.hand[player, i] < combo[0]:
                return False
            
        fold = [0, 0, 0, 1, 2]
        need_wings = (combo[2] - combo[1] + 1) * fold[combo[0]] if combo[3] > 0 else 0
        for i in range(15):
            if i < combo[1] or i > combo[2]:
                if self.hand[player, i] >= combo[3]:
                    need_wings -= 1
        if need_wings > 0:
            return False
        return True
    
    def getPossessMask(self, player):
        # 返回 player 拥有的牌型 mask
        mask = np.zeros(combo_cnt)
        for i in range(combo_cnt):
            if self.possess(player, combo_list[i]) == True:
                mask[i] = 1
        return mask
    
    def getMask1(self, player, combo):
        # getPossessMask 和 getComboMask 取交集
        return self.getPossessMask(player) * getComboMask(combo)
    
    def getMask2(self, player, combo, already_played):
        # 带翼的 mask，哪些翼是可以打的？
        # mask 的大小是 15 或者 13, 表示 15 种单牌和 13 种对子
        # 指明 combo 后：(1)少于1/2张的不能打 (2)和主体部分重复的不能打 (3)打过的不能打
        mask = np.ones(15 if combo[3] == 1 else 13)
        for i in range(mask.shape[0]):
                if self.hand[player, i] < combo[3]:
                    mask[i] = 0
        mask[range(combo[1], combo[2] + 1)] = 0
        mask[already_played] = 0
        return mask
        
    def getInput(self, player, combos):
        global _input_size
        
        p1 = (player + 1) % 3
        p2 = (player + 2) % 3
        '''
        myhand = np.zeros((4, 15))
        othershand = np.zeros((4, 15))
        for i in range(4):
            myhand[i, np.where(self.hand[player] == i + 1)] = 1
            othershand[i, np.where(self.hand[p1] + self.hand[p2] == i + 1)] = 1
        
        played_cards = np.zeros((3, 4, 15))
        p_list = [player, p1, p2]
        for i in range(3):
            p = p_list[i]
            for j in range(4):
                played_cards[i, j, np.where(self.initial_hand[p] - self.hand[p] == j + 1)] = 1
                
        handcnt = np.zeros((3, 20))
        for player in range(3):
            handcnt[player, range(np.sum(self.hand[player]))] = 1
        
        Input = np.concatenate([myhand.flatten(),
                               othershand.flatten(),
                               played_cards.flatten(),
                               handcnt.flatten()
#                               self.getPossessMask(player)
                              ])
        '''
        # 我手里有每种数值的牌多少张，对手还有每种数值的牌多少张没有出过
        Input = np.concatenate([np.array(self.hand[player]),
                                np.array(self.hand[p1]) + np.array(self.hand[p2])])
        # 我们三个人各打过每个数值多少张牌，各还有多少张牌
        for p in range(3):
            Input = np.concatenate([Input,
                                    np.array(self.initial_hand[p]) - np.array(self.hand[p]),
                                    np.array([np.sum(self.hand[p])])
                                   ])
        def getComboArray(combo):
            tmp = np.zeros(15)
            for i in range(combo[1], combo[2] + 1):
                tmp[i] += combo[0]
            return tmp
        # 前两回合那两个人各出了什么牌型的牌（只记录主体）
        for k in range(1, 2):
            for i in range(1, 3):
                Input = np.concatenate([Input,
                                        getComboArray(combos[(player + i) % 3][-k] if len(combos[(player + i) % 3]) >= k else (0, 0, 0, 0))
                                    ])
        # 我还有的牌型
        Input = np.concatenate([Input, self.getPossessMask(player)])
        # 我的手牌的差分(12张可连顺的部分)
        tmp = np.zeros(11)
        for i in range(11):
            tmp[i] = self.hand[player, i + 1] - self.hand[player, i]
        Input = np.concatenate([Input, tmp])
        _input_size = Input.shape[0]
        return Input

## 定义数据集类

In [31]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.X = []
        self.M = []
        self.Y = []
    def append(self, x, m, y):
        self.X.append(x)
        self.M.append(m)
        self.Y.append(y)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return (self.X[idx], self.M[idx], self.Y[idx])


In [36]:
DS1, DS2 = np.load('data.npz', allow_pickle=True)['DATA']
_input_size = DS1[0][0][0].shape[0]


KeyboardInterrupt: 

In [32]:
DS1 = [MyDataset(), MyDataset(), MyDataset()]
DS2 = [MyDataset(), MyDataset()]
# DS1 是三个玩家分别的出牌数据集，可能处于不同位置会有不同的出牌策略所以分别训练了网络
# DS2 是带牌数据集，分别是单排和顺子，因为数据量比较少而且感觉三个人没什么太大区别就合并到一起了

In [33]:
file_cnt = 0

for file in all_json:
#    print(file)
    with open(file, 'r+') as f:
        while True:
            data = f.readline()
            if(len(data) == 0):
                break
            try:
                data = json.loads(data)
                initdata = json.loads(data['initdata'])

                nb_bot = [0, 0, 0]
                exists_nb = 0
                for i in range(3):
                    if "bot" in data['players'][i].keys() and check_nb(data['players'][i]['bot']):
                        nb_bot[i] = 1
                        exists_nb = 1
                if exists_nb == 0:
                    break

                g = Game(initdata['allocation'])
                log = data['log']

                combos = [[(0, 0, 0, 0)], [(0, 0, 0, 0)], [(0, 0, 0, 0)]]
                las_play = -1

                for i in range(1, len(log), 2):
                    player = -1
                    for p in range(3):
                        if str(p) in log[i]:
                            player = p
                            break

                    if log[i][str(p)]['verdict'] != 'OK':
                        print(log[i][str(p)]['verdict'])
                        break

                    cards = log[i][str(player)]['response']
                    combos[player].append(getCombo(cards))
                    mainbody, bywings = getPartition(cards)
                    las_combo = (0, 0, 0, 0)
                    for i in range(1, 3):
                        if combos[(player + i) % 3][-1] != (0, 0, 0, 0):
                            las_combo = combos[(player + i) % 3][-1]

                    if nb_bot[player]:
                        input_x = g.getInput(player, combos)
                        input_m = g.getMask1(player, las_combo)
                        output_y = combo_dict[combos[player][-1]]
                        assert input_m[output_y] == 1
                        if np.sum(input_m) > 1:
                            DS1[player].append(input_x, input_m, output_y)

                    g.play(player, mainbody)

                    if len(bywings) != 0:
                        already_played = []
                        for w in bywings:
                            assert len(w) == 1 or len(w) == 2
                            if nb_bot[player]:
                                input_x = g.getInput(player, combos)
                                input_m = g.getMask2(player, combos[player][-1], already_played)
                                output_y = getCardId(w[0])
                                assert input_m[output_y] == 1
                                DS2[0 if len(w) == 1 else 1].append(input_x, input_m, output_y)

                            g.play(player, w)
                            already_played.append(getCardId(w[0]))                    
                            
            except:
                continue
    
    file_cnt += 1
    if file_cnt % 1 == 0:
        print("files = %d, lengths = (%d, %d, %d, %d, %d)"
              % (file_cnt, len(DS1[0]), len(DS1[1]), len(DS1[2]), len(DS2[0]), len(DS2[1])))


files = 1, lengths = (0, 0, 0, 0, 0)
files = 2, lengths = (0, 0, 0, 0, 0)
files = 3, lengths = (0, 0, 0, 0, 0)
files = 4, lengths = (0, 0, 0, 0, 0)
files = 5, lengths = (0, 0, 0, 0, 0)
files = 6, lengths = (0, 0, 0, 0, 0)
files = 7, lengths = (0, 0, 0, 0, 0)
files = 8, lengths = (0, 0, 0, 0, 0)
files = 9, lengths = (0, 0, 0, 0, 0)
files = 10, lengths = (0, 0, 0, 0, 0)
files = 11, lengths = (0, 0, 0, 0, 0)
files = 12, lengths = (0, 0, 0, 0, 0)
files = 13, lengths = (0, 0, 0, 0, 0)
files = 14, lengths = (0, 0, 0, 0, 0)
files = 15, lengths = (0, 0, 0, 0, 0)
files = 16, lengths = (0, 0, 0, 0, 0)
files = 17, lengths = (0, 0, 0, 0, 0)
files = 18, lengths = (0, 0, 0, 0, 0)
files = 19, lengths = (0, 0, 0, 0, 0)
files = 20, lengths = (0, 0, 0, 0, 0)
files = 21, lengths = (0, 0, 0, 0, 0)
files = 22, lengths = (0, 0, 0, 0, 0)
files = 23, lengths = (0, 0, 0, 0, 0)
files = 24, lengths = (0, 0, 0, 0, 0)
files = 25, lengths = (0, 0, 0, 0, 0)
files = 26, lengths = (0, 0, 0, 0, 0)
files = 27, lengths =

In [34]:
print("_input_size = ", _input_size)
print(len(DS1[0]), len(DS1[1]), len(DS1[2]), len(DS2[0]), len(DS2[1]))

_input_size =  498
29680 26425 27414 5166 2189


In [28]:
np.savez('data', DATA=(DS1, DS2))

/Users/lijiaheng/opt/anaconda3/envs/torch/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [29]:
DS10, DS20 = np.load('data.npz', allow_pickle=True)['DATA']

In [31]:
print(len(DS10[0]), len(DS10[1]), len(DS10[2]), len(DS20[0]), len(DS20[1]))

29680 26425 27414 5166 2189


如果读的是aigame的第一个bot，那么五个数据集的大小应该分别是 23785 20027 21129 6697 957

## 网络框架

In [35]:
HIDDEN_SIZE = 1024
torch.set_default_tensor_type(torch.DoubleTensor)

class MyModule(nn.Module):
    def __init__(self, INPUT_SIZE, OUTPUT_SIZE):
        super(MyModule, self).__init__()
        self.fc = nn.Sequential(OrderedDict([
                ('fc1', nn.Linear(INPUT_SIZE, HIDDEN_SIZE)),
                ('dropout', nn.Dropout(p = 0.5)),
                ('bn', nn.BatchNorm1d(HIDDEN_SIZE)),
                ('relu', nn.ReLU()),
                ('fc2', nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)),
                # ('dropout2', nn.Dropout(p = 0.5)),
                # ('bn2', nn.BatchNorm1d(OUTPUT_SIZE)),
            ]))
        
    def forward(self, x, m):
        return self.fc(x) * m
    

## 模型训练（主体）

In [36]:
best_acc = 0

def getPred(output):
    return output.detach().numpy().argmax(axis = 1)

def train(net, data_loader, data_size, criterion, optimizer):
    net.train()
    for i, (x, m, y) in enumerate(data_loader):
        optimizer.zero_grad()
        output = net(x, m)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
    total_correct = 0
    avg_loss = 0.0
    for i, (x, m, y) in enumerate(data_loader):
        output = net(x, m)
        avg_loss += criterion(output, y).sum()
        pred = getPred(output)
        total_correct += (pred == y.detach().numpy()).sum()
    avg_loss /= data_size
    cur_acc = float(total_correct) / data_size
    print('Training Avg. Loss: %f, Accuracy: %f' % (avg_loss, cur_acc))

def validate(net, data_loader, data_size, criterion, model_name):
    global best_acc
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (x, m, y) in enumerate(data_loader):
        output = net(x, m)
        avg_loss += criterion(output, y).sum()
        pred = getPred(output)
        total_correct += (pred == y.detach().numpy()).sum()
    
    avg_loss /= data_size
    cur_acc = float(total_correct) / data_size
    print('Validation Avg. Loss: %f, Accuracy: %f' % (avg_loss, cur_acc))
    
    if cur_acc > best_acc:
        best_acc = cur_acc
        torch.save(net, './model/best_model_for_' + model_name + '.pt')


In [37]:
for i in range(3):
        
    print("-----------------------------------------")
    print("Training model %d" % i)
    print("-----------------------------------------")

    net = MyModule(_input_size, combo_cnt)
    print(net)
    best_acc = 0
    
    train_size = int(0.95 * len(DS1[i]))
    valid_size = len(DS1[i]) - train_size
    print("train_size = ", train_size, "valid_size = ", valid_size)


    train_data, valid_data = torch.utils.data.random_split(DS1[i], [train_size, valid_size])

    train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 512)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = 512)

    for epoch in range(50):
        print("epoch = ", epoch)
        train(net, data_loader = train_loader,
              data_size = train_size,
              criterion = nn.CrossEntropyLoss(),
              optimizer = optim.Adam(net.parameters(), lr = 3e-4))
#              optimizer = optim.SGD(net.parameters(), lr = 2e-3, momentum=0.9))
        validate(net, data_loader = valid_loader,
                 data_size = valid_size,
                 criterion = nn.CrossEntropyLoss(),
                 model_name = str(i) + "mainbody")

-----------------------------------------
Training model 0
-----------------------------------------
MyModule(
  (fc): Sequential(
    (fc1): Linear(in_features=498, out_features=1024, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (fc2): Linear(in_features=1024, out_features=379, bias=True)
  )
)
train_size =  28196 valid_size =  1484
epoch =  0
Training Avg. Loss: 0.003739, Accuracy: 0.469889
Validation Avg. Loss: 0.007201, Accuracy: 0.469677
epoch =  1
Training Avg. Loss: 0.002518, Accuracy: 0.635019
Validation Avg. Loss: 0.006083, Accuracy: 0.633423
epoch =  2
Training Avg. Loss: 0.002092, Accuracy: 0.702795
Validation Avg. Loss: 0.005099, Accuracy: 0.681941
epoch =  3
Training Avg. Loss: 0.001834, Accuracy: 0.741559
Validation Avg. Loss: 0.004286, Accuracy: 0.727089
epoch =  4
Training Avg. Loss: 0.001636, Accuracy: 0.770003
Validation Avg. Loss: 0.003686, Ac

KeyboardInterrupt: 

## 模型训练（带翼）

In [14]:
for i in range(2):
    
    print("-----------------------------------------")
    print("Training model %d" % i)
    print("-----------------------------------------")
    
    net = MyModule(_input_size, 15 if i == 0 else 13)
    best_acc = 0

    train_size = int(0.95 * len(DS2[i]))
    valid_size = len(DS2[i]) - train_size
    print("train_size = ", train_size, "valid_size = ", valid_size)

    train_data, valid_data = torch.utils.data.random_split(DS2[i], [train_size, valid_size])

    train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 64)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = 64)

    for epoch in range(50):
        print("epoch = ", epoch)
        train(net, data_loader = train_loader,
               data_size = train_size,
               criterion = nn.CrossEntropyLoss(),
               optimizer = optim.Adam(net.parameters(), lr = 0.01))
        validate(net, data_loader = valid_loader,
                  data_size = valid_size,
                  criterion = nn.CrossEntropyLoss(),
                  model_name = str(i) + "bywings")

-----------------------------------------
Training model 0
-----------------------------------------
train_size =  4907 valid_size =  259
epoch =  0
Training Avg. Loss: 0.008086, Accuracy: 0.826370
Validation Avg. Loss: 0.013120, Accuracy: 0.810811
epoch =  1
Training Avg. Loss: 0.005591, Accuracy: 0.878133
Validation Avg. Loss: 0.007780, Accuracy: 0.880309
epoch =  2
Training Avg. Loss: 0.004262, Accuracy: 0.913593
Validation Avg. Loss: 0.008677, Accuracy: 0.884170
epoch =  3
Training Avg. Loss: 0.004810, Accuracy: 0.900754
Validation Avg. Loss: 0.009119, Accuracy: 0.864865
epoch =  4
Training Avg. Loss: 0.003322, Accuracy: 0.927858
Validation Avg. Loss: 0.009354, Accuracy: 0.864865
epoch =  5
Training Avg. Loss: 0.002821, Accuracy: 0.939067
Validation Avg. Loss: 0.007050, Accuracy: 0.907336
epoch =  6
Training Avg. Loss: 0.002482, Accuracy: 0.949256
Validation Avg. Loss: 0.007523, Accuracy: 0.891892
epoch =  7
Training Avg. Loss: 0.002991, Accuracy: 0.937029
Validation Avg. Loss: 0.0

Training Avg. Loss: 0.001878, Accuracy: 0.981722
Validation Avg. Loss: 0.018236, Accuracy: 0.918182
epoch =  22
Training Avg. Loss: 0.000943, Accuracy: 0.990380
Validation Avg. Loss: 0.026373, Accuracy: 0.927273
epoch =  23
Training Avg. Loss: 0.001659, Accuracy: 0.984608
Validation Avg. Loss: 0.019915, Accuracy: 0.945455
epoch =  24
Training Avg. Loss: 0.000685, Accuracy: 0.993266
Validation Avg. Loss: 0.021277, Accuracy: 0.936364
epoch =  25
Training Avg. Loss: 0.002628, Accuracy: 0.975469
Validation Avg. Loss: 0.043354, Accuracy: 0.918182
epoch =  26
Training Avg. Loss: 0.000829, Accuracy: 0.991823
Validation Avg. Loss: 0.029636, Accuracy: 0.936364
epoch =  27
Training Avg. Loss: 0.002707, Accuracy: 0.979798
Validation Avg. Loss: 0.043997, Accuracy: 0.927273
epoch =  28
Training Avg. Loss: 0.000945, Accuracy: 0.993266
Validation Avg. Loss: 0.035893, Accuracy: 0.945455
epoch =  29
Training Avg. Loss: 0.000791, Accuracy: 0.991342
Validation Avg. Loss: 0.040436, Accuracy: 0.936364
epoc

## main.py

In [15]:
os.environ.get("USER", "") == "root"

False

In [31]:
_BOTZONE_ONLINE = os.environ.get("USER", "") == "root"

my_hand = []
g = Game([[], [], []])
my_pos = -1
others = []
combos = [(0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0)]
las_combo = (0, 0, 0, 0)

def BIDDING():
    bid_val = 0
    print(json.dumps({
        "response": bid_val
    }))
    if not _BOTZONE_ONLINE:
        assert(0)
    exit()

model_path = "./data/fightlandlord_model/" if _BOTZONE_ONLINE else "./model/"

def PLAYING():
    def getFromHand(idx):
        global my_hand
        for c in my_hand:
            if getCardId(c) == idx:
                my_hand.remove(c)
                return c
    
    to_play = []
        
    model_name = "best_model_for_" + str(my_pos) + "mainbody.pt"
    model = torch.load(model_path + model_name)
    
    mask = g.getMask1(my_pos, las_combo)
    combo_id = -1
    if np.sum(mask) == 1:
        combo_id = np.argmax(mask)
    else:
        combo_id = model(torch.from_numpy(g.getInput(my_pos, combos)).unsqueeze(0),
                         torch.from_numpy(mask)).unsqueeze(0).detach().numpy().argmax()
    
    combo = combo_list[combo_id]
    for i in range(combo[1], combo[2] + 1):
        for j in range(combo[0]):
            to_play.append(getFromHand(i))
    g.play(my_pos, to_play)
    
    if combo[3] != 0:
        model_name = "best_model_for_" + str(combo[3] - 1) + "bywings.pt"
        model = torch.load(model_path + model_name)

        cnt = (combo[2] - combo[1] + 1) * (1 if combo[0] == 3 else 2)
        already_played = []
        for i in range(cnt):
            wing_id = model(torch.from_numpy(g.getInput(my_pos, combos)),
                            torch.from_numpy(g.getMask2(my_pos, combo, already_played))).detach().numpy().argmax()
            tmp = []
            if wing_id < 15:
                tmp = [getFromHand(wing_id)]
            else:
                wing_id -= 15
                tmp = [getFromHand(wing_id), getFromHand(wing_id)]
            g.play(my_pos, tmp)
            to_play.extend(tmp)
            already_played.append(wing_id)
    
    print(json.dumps({
        "response": to_play
    }))
    if not _BOTZONE_ONLINE:
        assert 0
    exit()
    
if __name__ == "__main__":
    initCombo()
    data = json.loads(input())
    my_hand, others_hand = data["requests"][0]["own"], []
    for i in range(54):
        if i not in my_hand:
            others_hand.append(i)

    TODO = "bidding"
    if "bid" in data["requests"][0]:
        bid_list = data["requests"][0]["bid"]
    
    for i in range(len(data["requests"])):
        request = data["requests"][i]
        
        if "publiccard" in request:
            bot_pos = request["pos"]
            lord_pos = request["landlord"]
            my_pos = (bot_pos - lord_pos + 3) % 3
            others = [(my_pos + 1) % 3, (my_pos + 2) % 3]
            tmp = [[], [], []]
            if my_pos == 0:
                my_hand.extend(request["publiccard"])
                tmp[0] = my_hand
                tmp[1], tmp[2] = others_hand[:17], others_hand[17:] # 随便分
            else:
                tmp[my_pos] = my_hand
                tmp[0] = others_hand[:20]
                tmp[2 if my_pos == 1 else 1] = others_hand[20:]
            g = Game(tmp)
            
        if "history" in request:
            history = request["history"]
            TODO = "playing"
            for j in range(2):
                p = others[j]
                cards = history[j]
                g.play(p, cards)
                combos[p] = getCombo(cards)

            if i < len(data["requests"]) - 1:
                cards = data["responses"][i]
                g.play(my_pos, cards)
                for c in cards:
                    my_hand.remove(c)
                combos[my_pos] = getCombo(cards)
    
    for i in range(1, 3):
        if combos[(my_pos + i) % 3] != (0, 0, 0, 0):
            las_combo = combos[(my_pos + i) % 3]

    if TODO == "bidding":
        BIDDING()
    else:
        PLAYING()

{"requests":[{"own":[34,48,26,6,46,37,51,36,1,27,17,5,24,29,18,52,53],"bid":[0,0]},{"history":[[7,9,10,11],[4,32,33,35]],"own":[34,48,26,6,46,37,51,36,1,27,17,5,24,29,18,52,53],"publiccard":[39,0,9],"landlord":0,"pos":2,"finalbid":1},{"history":[[0,44,45,47],[]]},{"history":[[],[]]}],"responses":[0,[],[52,53]]}
{"response": [26, 27, 24, 1]}


AssertionError: 